In [1]:
# Upload Database
from google.colab import files
uploaded = files.upload()

Saving heart.csv to heart.csv


In [2]:
import pandas as pd

df = pd.read_csv('heart.csv')

In [3]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [5]:
# Menghitung jumlah target yang memiliki nilai 1
postive_disease = len(df[df['target'] == 1])

# Menghitung jumlah target yang memiliki nilai 0
negative_disease = len(df[df['target'] == 0])
print(postive_disease)
print(negative_disease)

526
499


In [7]:
from imblearn.over_sampling import SMOTE
from collections import Counter
import pandas as pd


# Pisahkan fitur (X) dan label (y)
X = df.drop(columns=['target'])
y = df['target']

# Cek distribusi kelas sebelum oversampling
print("Distribusi kelas sebelum oversampling:", Counter(y))

# Buat objek SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)

# Terapkan SMOTE untuk oversampling
X_resampled, y_resampled = smote.fit_resample(X, y)

# Cek distribusi kelas setelah oversampling
print("Distribusi kelas setelah oversampling:", Counter(y_resampled))

Distribusi kelas sebelum oversampling: Counter({1: 526, 0: 499})
Distribusi kelas setelah oversampling: Counter({0: 526, 1: 526})


In [8]:
# Splitting data menjadi data training dan data testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [10]:
# Normalisasis data dengan standar scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# membangun model dengan parameter C, gamma, dan kernel
model = SVC()
parameters = {
    'kernel': ['rbf'],
    'C':     [0.1, 1, 10, 100],
    'gamma': [0.005, 0.05, 0.5, 5, 50]
}
grid_search = GridSearchCV(model, parameters)

# melatih model dengan fungsi fit
grid_search.fit(X_train_normalized,y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [0.005, 0.05, 0.5, 5, 50],
                         'kernel': ['rbf']})

In [13]:
# menampilkan parameter terbaik dari objek grid_search
print(grid_search.best_params_)

{'C': 100, 'gamma': 0.05, 'kernel': 'rbf'}


In [18]:
from sklearn.svm import SVC

# membuat objek SVC dan memanggil fungsi fit untuk melatih model
model_svm = SVC(C=100, gamma=0.05, kernel='rbf')
model_svm.fit(X_train_normalized, y_train)

SVC(C=100, gamma=0.05)

In [20]:
X_test_normalized = scaler.transform(X_test)

In [21]:
# Menampilkan skor akurasi prediksi
model_svm.score(X_test_normalized, y_test)

0.995260663507109

In [23]:
import joblib

# Simpan model dan scaler ke dalam file
joblib.dump(model_svm, 'model_svm.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [25]:
import joblib

# Memuat model dan scaler dari file
loaded_model = joblib.load('model_svm.pkl')
loaded_scaler = joblib.load('scaler.pkl')

In [28]:
import numpy as np
data_tunggal = np.array([[57,	0,	0, 128,	303,	0,	0,	159,	0,	0.0,	2,	1,	2]])
X_test_normalized = loaded_scaler.transform(data_tunggal)
# Lakukan prediksi
classify = loaded_model.predict(X_test_normalized)
classify

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([1])

In [29]:
import numpy as np
data_tunggal = np.array([[52,	1,	0,	125,	212,	0,	1,	168,	0,	1.0,	2,	2,	3	]])
X_test_normalized = loaded_scaler.transform(data_tunggal)
# Lakukan prediksi
classify = loaded_model.predict(X_test_normalized)
classify

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([0])